In [1]:
import os
class_names = ['cothe', 'khong', 'nguoi', 'nhung', 'toi']
states = [12, 9, 9, 9, 6]

length = 0
for d in class_names:
    length += len(os.listdir("data/" + d))
print(length)

607


In [2]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm

def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    # mfcc is 12 x T matrix
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
    # delta feature 1st order and 2nd order
    delta1 = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)
    # X is 36 x T
    X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
    # return T x 36 (transpose of X)
    return X.T # hmmlearn use T x N matrix


In [3]:
all_data = {}
all_labels = {}
for cname in class_names:
    file_paths = [os.path.join("data", cname, i) for i in os.listdir(os.path.join('data', cname)) if i.endswith('.wav')]
    data = [get_mfcc(file_path) for file_path in file_paths]
    all_data[cname] = data
    all_labels[cname] = [class_names.index(cname) for i in range(len(file_paths))]

C:\Users\Cong\AppData\Local\Temp\ipykernel_2040\855361948.py:13: FutureWarning: Pass y=[ 1.6784668e-04  6.1035156e-05 -6.1035156e-05 ... -3.1784058e-02
 -5.2185059e-02 -2.5436401e-02], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(
C:\Users\Cong\AppData\Local\Temp\ipykernel_2040\855361948.py:13: FutureWarning: Pass y=[ 0.20503235  0.2868805   0.23847961 ... -0.12065125 -0.11172485
 -0.06710815], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(
C:\Users\Cong\AppData\Local\Temp\ipykernel_2040\855361948.py:13: FutureWarning: Pass y=[-5.1235500e-05  6.8688394e-05  1.7983712e-05 ...  2.2450263e-04
 -1.0624271e-03  0.0000000e+00], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(
C:\Users\Cong\AppData\Local\Temp\ipykernel_2040\8553

In [4]:
from sklearn.model_selection import train_test_split

X = {'train': {}, 'test': {}}
y = {'train': {}, 'test': {}}
for cname in class_names:
    x_train, x_test, _, y_test = train_test_split(
        all_data[cname], all_labels[cname], 
        test_size = 0.33, 
        random_state=42
    )
    X['train'][cname] = x_train
    X['test'][cname] = x_test
    y['test'][cname] = y_test

In [5]:
for cname in class_names:
    print(cname,len(X['train'][cname]), len(X['test'][cname]), len(y['test'][cname]))

cothe 97 48 48
khong 66 33 33
nguoi 94 47 47
nhung 66 33 33
toi 79 40 40


In [6]:
import hmmlearn.hmm as hmm

model = {}
for idx, cname in enumerate(class_names):
    start_prob_ = np.full(states[idx], 0.0)
    start_prob_[0] = 1.0
    trans_matrix = np.full((states[idx], states[idx]), 0.0)
    p = 0.5
    np.fill_diagonal(trans_matrix, p)
    np.fill_diagonal(trans_matrix[0:, 1:], 1 - p)
    trans_matrix[-1, -1] = 1.0
    
    #trans matrix
    print(cname)
    print(trans_matrix) 

    model[cname] = hmm.GaussianHMM(
        n_components=states[idx], 
        verbose=True, 
        n_iter=300, 
        startprob_prior=start_prob_, 
        transmat_prior=trans_matrix,
        params='stmc',
        init_params='mc',
        random_state=42
    )

    model[cname].fit(X=np.vstack(X['train'][cname]), lengths=[x.shape[0] for x in X['train'][cname]])

cothe
[[0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. ]]


         1     -364864.3382             +nan
         2     -350620.0524      +14244.2858
         3     -348514.2098       +2105.8426
         4     -347821.2642        +692.9457
         5     -347498.1609        +323.1033
         6     -347266.4823        +231.6786
         7     -347101.3869        +165.0954
         8     -346981.3547        +120.0322
         9     -346901.3900         +79.9647
        10     -346826.5893         +74.8007
        11     -346744.6103         +81.9790
        12     -346648.2314         +96.3789
        13     -346543.8807        +104.3507
        14     -346463.0279         +80.8529
        15     -346409.1471         +53.8807
        16     -346366.8361         +42.3110
        17     -346315.2732         +51.5629
        18     -346291.4857         +23.7875
        19     -346261.9141         +29.5716
        20     -346234.5651         +27.3490
        21     -346213.3558         +21.2093
        22     -346195.5049         +17.8508
        23

khong
[[0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. ]]


         1     -411224.7546             +nan
         2     -310635.1018     +100589.6528
         3     -201077.0096     +109558.0922
         4      -56066.0245     +145010.9851
         5       18925.7407      +74991.7651
         6       27812.8915       +8887.1508
         7       30758.5548       +2945.6633
         8       31200.8471        +442.2923
         9       31376.7348        +175.8877
        10       31406.8475         +30.1127
        11       31409.3417          +2.4942
        12       31478.1100         +68.7683
        13       31484.5493          +6.4393
        14       31488.6863          +4.1371
        15       31492.2283          +3.5419
        16       31495.8226          +3.5944
        17       31498.2011          +2.3784
        18       31499.7891          +1.5880
        19       31500.9088          +1.1197
        20       31501.5126          +0.6038
        21       31501.8195          +0.3070
        22       31502.0121          +0.1926
        23

nguoi
[[0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. ]]


         1     -216373.1687             +nan
         2     -206794.2378       +9578.9309
         3     -205685.1083       +1109.1294
         4     -205411.6667        +273.4417
         5     -205191.7802        +219.8865
         6     -205097.0212         +94.7590
         7     -205046.2691         +50.7520
         8     -205004.4505         +41.8187
         9     -204973.7194         +30.7310
        10     -204937.5901         +36.1293
        11     -204900.3570         +37.2331
        12     -204859.9425         +40.4145
        13     -204817.0268         +42.9157
        14     -204777.8094         +39.2174
        15     -204712.3010         +65.5084
        16     -204639.5807         +72.7204
        17     -204575.0013         +64.5794
        18     -204509.8490         +65.1523
        19     -204472.1665         +37.6824
        20     -204432.2148         +39.9517
        21     -204402.8382         +29.3767
        22     -204379.5545         +23.2837
        23

nhung
[[0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. ]]


         1     -137003.6767             +nan
         2     -129436.7881       +7566.8885
         3     -128517.0631        +919.7250
         4     -128254.1920        +262.8711
         5     -128184.2656         +69.9264
         6     -128142.2576         +42.0080
         7     -128126.4231         +15.8345
         8     -128123.4649          +2.9582
         9     -128122.0950          +1.3699
        10     -128120.7960          +1.2990
        11     -128120.1353          +0.6608
        12     -128119.7909          +0.3444
        13     -128119.5578          +0.2331
        14     -128119.3835          +0.1743
        15     -128119.2228          +0.1607
        16     -128119.0297          +0.1931
        17     -128118.7280          +0.3017
        18     -128118.1392          +0.5888
        19     -128117.2580          +0.8812
        20     -128116.7506          +0.5074
        21     -128116.3862          +0.3644
        22     -128115.9991          +0.3870
        23

toi
[[0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. ]]


         1     -187464.1308             +nan
         2     -177051.5972      +10412.5336
         3     -174917.5723       +2134.0250
         4     -174349.5571        +568.0152
         5     -173995.5646        +353.9925
         6     -173826.2902        +169.2743
         7     -173754.4036         +71.8866
         8     -173718.9027         +35.5009
         9     -173698.3573         +20.5454
        10     -173674.2696         +24.0877
        11     -173667.2125          +7.0571
        12     -173606.3811         +60.8314
        13     -173537.9344         +68.4467
        14     -173508.3080         +29.6264
        15     -173474.1238         +34.1842
        16     -173410.1163         +64.0075
        17     -173369.8110         +40.3053
        18     -173357.9176         +11.8934
        19     -173350.5589          +7.3587
        20     -173343.5353          +7.0236
        21     -173336.8937          +6.6416
        22     -173326.6643         +10.2295
        23

In [7]:
import pickle

# save model
for cname in class_names:
    name = f'models_train\model_{cname}.pkl'
    # name = f'models\model_{cname}.pkl'
    with open(name, 'wb') as file: 
        pickle.dump(model[cname], file)

In [8]:
import pickle, os
import numpy as np

from sklearn.metrics import classification_report

In [9]:
y_true = []
y_pred = []
for cname in class_names:
    for mfcc, target in zip(X['test'][cname], y['test'][cname]):
        scores = [model[cname].score(mfcc) for cname in class_names]
        pred = np.argmax(scores)
        y_pred.append(pred)
        y_true.append(target)
    print((np.array(y_true) == np.array(y_pred)).sum()/len(y_true))
print(y_true)
print(y_pred)


0.9583333333333334
0.9753086419753086
0.96875
0.968944099378882
0.9651741293532339
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [10]:
report = classification_report(y_true, y_pred, target_names=class_names)
print(report)

              precision    recall  f1-score   support

       cothe       0.92      0.96      0.94        48
       khong       1.00      1.00      1.00        33
       nguoi       0.96      0.96      0.96        47
       nhung       1.00      0.97      0.98        33
         toi       0.97      0.95      0.96        40

    accuracy                           0.97       201
   macro avg       0.97      0.97      0.97       201
weighted avg       0.97      0.97      0.97       201



In [11]:
#loadmodels
import pickle

model_train = {}
for key in class_names:
    name = f"models\model_{key}.pkl"
    with open(name, 'rb') as file:
        model_train[key] = pickle.load(file)

In [12]:
print(model_train)

{'cothe': GaussianHMM(init_params='mc', n_components=12, n_iter=300, random_state=42,
            startprob_prior=array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
            transmat_prior=array([[0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 

In [13]:
# import pickle as pkl
# import pandas as pd
# with open("model_cothe.pkl", "rb") as f:
#     object = pkl.load(f)
    
# df = pd.DataFrame(object)
# df.to_csv(r'models\model_cothe.csv')